# Simple VASP input file generation for MgO

In [1]:
from ase.build import bulk
from pytheos.vasp import inputs, submission, custodian

# Relaxation

First we need to generate an ase.Atoms object for a primitive (2-atom) unit cell of MgO

In [2]:
unitcell = bulk("MgO", crystalstructure="rocksalt", a=4.25, cubic=False)
print(unitcell)

Atoms(symbols='MgO', pbc=True, cell=[[0.0, 2.125, 2.125], [2.125, 0.0, 2.125], [2.125, 2.125, 0.0]])


Now we will make an object for VASP calculation inputs, specifying we want to turn spin polarization off (`ISPIN` = 1) since we know MgO is non-magnetic
- note that here we are using the `KSPACING` INCAR tag to control the k-point density spacing

In [3]:
calc_inputs = inputs.CalcInputs(structure=unitcell, incar_changes={"ISPIN": 1})

/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/io/vasp/sets.py:486: BadInputSetWarning: POTCAR data with symbol Mg_pv is not known by pymatgen to correspond with the selected user_potcar_functional='PBE_54'. This POTCAR is known to correspond with functionals ['PBE_64', 'PBE_54_W_HASH', 'PBE_52_W_HASH']. Please verify that you are using the right POTCARs!
  potcar="\n".join(self.potcar_symbols) if potcar_spec else self.potcar,
/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/io/vasp/sets.py:486: BadInputSetWarning: POTCAR data with symbol O is not known by pymatgen to correspond with the selected user_potcar_functional='PBE_54'. This POTCAR is known to correspond with functionals ['PBE_64', 'PBE_54_W_HASH', 'PBE_52_W_HASH']. Please verify that you are using the right POTCARs!
  potcar="\n".join(self.potcar_symbols) if potcar_spec else self.potcar,


We can visualize all of the necessary VASP input files by calling each attribute of the `calc_inputs` object

In [4]:
calc_inputs.incar

{'ALGO': 'All', 'EDIFF': 1e-06, 'EDIFFG': -0.02, 'ENAUG': 1360, 'ENCUT': 680.0, 'IBRION': 2, 'ISIF': 3, 'ISMEAR': 0, 'ISPIN': 1, 'KSPACING': 0.25, 'LAECHG': False, 'LASPH': True, 'LCHARG': True, 'LELF': False, 'LMIXTAU': True, 'LORBIT': 11, 'LREAL': 'Auto', 'LVTOT': False, 'LWAVE': True, 'METAGGA': 'R2scan', 'NELM': 500, 'NSW': 250, 'PREC': 'Accurate', 'SIGMA': 0.05, 'MAGMOM': [0, 0], 'ISYM': 0, 'KPAR': 1, 'NCORE': 32, 'POTIM': 0.25, 'TIME': 0.1}

In [5]:
calc_inputs.poscar

Mg1 O1
1.0
   0.0000000000000000    2.1250000000000000    2.1250000000000000
   2.1250000000000000    0.0000000000000000    2.1250000000000000
   2.1250000000000000    2.1250000000000000    0.0000000000000000
Mg O
1 1
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Mg
  -0.5000000000000000    0.5000000000000000    0.5000000000000000 O

In [6]:
calc_inputs.potcar

[PotcarSingle(symbol='Mg_pv', functional='PBE', TITEL='PAW_PBE Mg_pv 13Apr2007', VRHFIN='Mg: p6s2', n_valence_elec=8),
 PotcarSingle(symbol='O', functional='PBE', TITEL='PAW_PBE O 08Apr2002', VRHFIN='O: s2p4', n_valence_elec=6)]

Now we can write the input files to a new directory so we can run them on the cluster

In [ ]:
calc_inputs.write_files(output_dir="relax")

We can also quickly write SLURM submission scripts for simple VASP calculations on the Roar Collab HPC

In [ ]:
submission.write_vasp_submission(job_name="simple-MgO", output_dir="relax")

Finally, we can write a Custodian script to automate a double-relaxation calculation

In [9]:
custodian.write_double_relax_script(output_dir="relax")

We also could change the INCAR settings such that we use a `KPOINTS` file instead of the `KSPACING` flag...

In [10]:
calc_inputs = inputs.CalcInputs(structure=unitcell, incar_changes={"ISPIN": 1}, kpoint_mesh=(8, 8, 8))
calc_inputs.kpoints

Default gamma
0
Gamma
8 8 8

Or we could decide to run a `GGA` calculation instead of a `metaGGA` calculation...
- notice that the `METAGGA = R2SCAN` is no longer present, and VASP defaults to using `GGA = PE` (PBE)
- there are also some other changes to better align with the MPRelaxSet calculation settings

In [12]:
calc_inputs = inputs.CalcInputs(structure=unitcell, incar_changes={"ISPIN": 1}, mp_input_set="MPRelaxSet")
calc_inputs.incar
calc_inputs.write_files(output_dir="relax-gga")

/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/io/vasp/inputs.py:2508: UnknownPotcarWarning: POTCAR data with symbol Mg_pv is not known to pymatgen. Your POTCAR may be corrupted or pymatgen's POTCAR database is incomplete.
  return cls(file.read(), symbol=symbol or None)  # type:ignore[arg-type]
/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/io/vasp/sets.py:486: BadInputSetWarning: POTCAR data with symbol Mg_pv is not known by pymatgen to correspond with the selected user_potcar_functional='PBE'. This POTCAR is known to correspond with functionals []. Please verify that you are using the right POTCARs!
  potcar="\n".join(self.potcar_symbols) if potcar_spec else self.potcar,


You should obtain close to the following energies and lattice vectors for the GGA and metaGGA relaxations from the input files generated above ->. 

**GGA** (-5.934566595 eV/atom)
```
0.0000032250837997    2.1271822422093987    2.1271821963282123
2.1271830059650108    0.0000026913606158    2.1271827302543911
2.1271831229417244    2.1271828932130226    0.0000025744525814
```

**metaGGA** (-8.063949545 eV/atom)
```
-0.0000406845964276    2.0990635024096309    2.0990640627318644
2.0990637879703509   -0.0000411745636187    2.0990645520833455
2.0990635271133811    2.0990637310704008   -0.0000409131525682
```

# Density of States

We can utilize the pytheos.modifiers `CalcModifier` module to quickly generate vasp input files for a density of states (DOS) calculation as well

In [17]:
from pytheos.vasp import modifiers

calc = modifiers.CalcModifier(source_dir="relax-r2scan")
calc.to_dos(energy_window=8) # expanding the energy window around the Fermi energy since MgO has such a large band gap
calc.write_files(output_dir="dos-r2scan")

We can also write a static custodian script to run the DOS which is helpful in case some error arise during the calculation

In [18]:
custodian.write_script("dos-r2scan")

The resulting DOS can be plotted using `sumo` (https://github.com/SMTG-Bham/sumo) using the following command in the directory where the DOS was run

```
sumo-dosplot --format png --xmax 8
```

<img src="images/MgO_dos.png" alt="isolated" width="500"/>

The electronic band gap should be ~5.7185 eV when calculated with R2SCAN

## Band Structure

In [ ]:
calc = modifiers.CalcModifier(source_dir="relax-r2scan")
calc.to_bandstructure(sumo_kgen_cmd="sumo-kgen --pymatgen --hybrid --split 50000") # we only need the `--split 50000` since this is being run in a jupyter notebook to avoid the command line prompt from sumo...
calc.write_files(output_dir="band-r2scan")

Detected a meta-GGA calculation -> NSCF calculation will use `GGA = PE`


/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/seekpath/hpkot/__init__.py:172: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  conv_lattice = dataset["std_lattice"]
/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/pymatgen/symmetry/bandstructure.py:181: UserWarning: The input structure does not match the expected standard primitive! The path may be incorrect. Use at your own risk.
  kpath = KPathSetyawanCurtarolo(self._structure, symprec, angle_tolerance, atol)
Structure information:
/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/sumo/symmetry/kpoints.py:146: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  logging.info(f"\tSpace group number: {kpath._spg_data['number']}")
	Space group number: 225
/Users/jtsivak/miniconda3/envs/pytheos/lib/python3.12/site-packages/sumo/symmetry/kpath.py:227: DeprecationWarning: dict interface is deprecated. Use attribute i

Since this is a metaGGA calculation, note the extra steps needed to generate a bandstructure (see doc string for pytheos.vasp.modifiers.CalcModifier.to_dos())

Once we have gone through those steps we can generate the band structure using the following command in the directory where the BS was run

```
sumo-bandplot --format png --ymax 8 --dos ../dos-r2scan/vasprun.xml
```

<img src="images/MgO_band.png" alt="isolated" width="750"/>